In [1]:
import pickle
import numpy as np

In [2]:
file = "DeepMoji-master/data/PsychExp/raw.pickle"

In [3]:
data = pickle.load(open(file,'rb'),encoding='latin1')

In [4]:
data.keys()

dict_keys(['info', 'texts', 'val_ind', 'test_ind', 'train_ind'])

In [5]:
label = data['info']

In [6]:
texts = data['texts']

In [59]:
val_index = data['val_ind']

In [60]:
test_index = data['test_ind']

In [61]:
train_index = data['train_ind']

In [62]:
val_index[0]

4352

In [63]:
print(label[4352] , texts[4352])

{'label': array([0., 0., 1., 0., 0., 0., 0.])} When I saw a car run over a child, out of carelessness.


In [12]:
def glove_vec(path):
    import numpy as np
    with open(path,'r',encoding='utf-8') as f:
        words = set()
        word_vec = {}
        for line in f:
            line = line.strip().split()
            key = line[0]
            words.add(key)
            val = np.array(line[1:],dtype=np.float64)
            word_vec[key] = val
            
        # Dict (Vocab) --> set
        word2idx = {}
        idx2word = {}
        
        for i,w in enumerate(sorted(words)):
            word2idx[w] = i
            idx2word[i] = w
            
    return word_vec,word2idx,idx2word

In [39]:
word_vec,word2idx,idx2word = glove_vec('glove.6B/glove.6B.50d.txt')
vocab_size = len(word_vec)

In [40]:
vocab_size

400000

In [16]:
from nltk import word_tokenize

In [77]:
def get_XY_data(Index):
    X = np.zeros((len(Index) , 128 ))
    Y = np.zeros((len(Index) , 7))
    
    zer = np.zeros((50 ,))
    for i ,index in enumerate(Index):
        line = word_tokenize(texts[index].lower())
        
        #print(index)
        Y[i] = label[index]['label']
        for j , word  in  enumerate(line[:128]):
             X[i , j] =  word2idx.get(word , 0)
    
    return X ,Y

In [78]:
X_train , Y_train  = get_XY_data(train_index)
print(X_train.shape , Y_train.shape)

X_test , Y_test  = get_XY_data(test_index)
print(X_test.shape , Y_test.shape)

X_val , Y_val  = get_XY_data(val_index)
print(X_val.shape , Y_val.shape)


(900, 128) (900, 7)
(6480, 128) (6480, 7)
(100, 128) (100, 7)


In [79]:
from keras.layers import Embedding , Flatten ,Dense , Bidirectional ,LSTM
from keras.models import Sequential

In [80]:


model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=128))
model.add(Bidirectional(LSTM(128)))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 128, 50)           20000000  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               183296    
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1799      
Total params: 20,185,095
Trainable params: 20,185,095
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
model.predict(X_train[0])

array([[0.14201638, 0.14288639, 0.14181702, 0.14222808, 0.14324875,
        0.14416346, 0.14363983],
       [0.14374709, 0.14279792, 0.14387421, 0.14279208, 0.14169517,
        0.14244094, 0.14265256],
       [0.14311263, 0.14213657, 0.14232287, 0.14232603, 0.1428991 ,
        0.14273866, 0.14446412],
       [0.14223164, 0.14289026, 0.1431509 , 0.14364623, 0.1427494 ,
        0.14296745, 0.1423641 ],
       [0.14202522, 0.14329107, 0.14250772, 0.14231656, 0.14338285,
        0.14421962, 0.14225693],
       [0.14295256, 0.1418416 , 0.14305754, 0.14280866, 0.1426298 ,
        0.14314488, 0.14356494],
       [0.14336945, 0.1429956 , 0.14165287, 0.14225993, 0.14276905,
        0.14412022, 0.1428329 ],
       [0.1434992 , 0.14325027, 0.14301878, 0.14189154, 0.14319955,
        0.14272243, 0.14241822],
       [0.14223164, 0.14289026, 0.1431509 , 0.14364623, 0.1427494 ,
        0.14296745, 0.1423641 ],
       [0.14414805, 0.14239877, 0.1423076 , 0.14307176, 0.1418484 ,
        0.14324445, 0.1

In [84]:
history = model.fit(X_train , Y_train , batch_size = 100 , epochs=100 , validation_data=(X_val,Y_val))

Epoch 1/100
9/9 [==============================] - 3s 385ms/step - loss: 1.6119 - accuracy: 0.3789 - val_loss: 1.8974 - val_accuracy: 0.2100
Epoch 2/100
9/9 [==============================] - 4s 417ms/step - loss: 1.5033 - accuracy: 0.4156 - val_loss: 1.8752 - val_accuracy: 0.2400
Epoch 3/100
9/9 [==============================] - 4s 412ms/step - loss: 1.3803 - accuracy: 0.5111 - val_loss: 1.8596 - val_accuracy: 0.3000
Epoch 4/100
9/9 [==============================] - 4s 432ms/step - loss: 1.6879 - accuracy: 0.5556 - val_loss: 2.0362 - val_accuracy: 0.2700
Epoch 5/100
9/9 [==============================] - 4s 405ms/step - loss: 1.3620 - accuracy: 0.5200 - val_loss: 2.0154 - val_accuracy: 0.2700
Epoch 6/100
9/9 [==============================] - 4s 428ms/step - loss: 1.1992 - accuracy: 0.5544 - val_loss: 2.0059 - val_accuracy: 0.2700
Epoch 7/100
9/9 [==============================] - 4s 407ms/step - loss: 1.0304 - accuracy: 0.6922 - val_loss: 1.9561 - val_accuracy: 0.3000
Epoch 8/100
9

Epoch 59/100
9/9 [==============================] - 4s 402ms/step - loss: 0.0130 - accuracy: 0.9978 - val_loss: 4.1681 - val_accuracy: 0.3600
Epoch 60/100
9/9 [==============================] - 4s 402ms/step - loss: 0.0118 - accuracy: 0.9989 - val_loss: 4.1950 - val_accuracy: 0.3500
Epoch 61/100
9/9 [==============================] - 4s 421ms/step - loss: 0.0110 - accuracy: 0.9989 - val_loss: 4.2783 - val_accuracy: 0.3400
Epoch 62/100
9/9 [==============================] - 4s 405ms/step - loss: 0.0102 - accuracy: 0.9989 - val_loss: 4.3231 - val_accuracy: 0.3400
Epoch 63/100
9/9 [==============================] - 4s 428ms/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 4.3699 - val_accuracy: 0.3400
Epoch 64/100
9/9 [==============================] - 4s 401ms/step - loss: 0.0092 - accuracy: 0.9989 - val_loss: 4.4168 - val_accuracy: 0.3300
Epoch 65/100
9/9 [==============================] - 4s 428ms/step - loss: 0.0086 - accuracy: 0.9989 - val_loss: 4.4684 - val_accuracy: 0.3300
Epoch 

In [85]:
model.evaluate(X_test,Y_test)

203/203 [==============================] - 9s 42ms/step - loss: 4.7417 - accuracy: 0.3020


[4.741705417633057, 0.30200618505477905]